#Configuración

In [25]:
!pip install simpletransformers

#2-Preprocesamineto de los datos anotados

In [ ]:
%cd /content/drive/MyDrive/proyecto/ner/Webapp-ner

/content/drive/MyDrive/proyecto/ner/Webapp-ner


In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
import pandas as pd
test_data = pd.read_csv("data/dataset/test.csv")
train_data = pd.read_csv("data/dataset/train.csv")

In [ ]:
test_data

,sentence_id,words,labels
0,2573,El,I-LOC
1,655,Kupa,I-ORG
2,738,.,O
3,4398,Kantarawi,B-LOC
4,2177,1994,I-ORG
...,...,...,...
3506,3835,Herrera,I-LOC
3507,5082,Porras,I-PER
3508,5003,Ian,B-PER
3509,1810,(,O


In [ ]:
train_data

,sentence_id,words,labels
0,4836,Matías,I-PER
1,489,PUSAPUNA,O
2,4164,Domingo,I-LOC
3,774,suyukuna,I-ORG
4,4746,Begoña,B-PER
...,...,...,...
14035,483,kawallu,I-ORG
14036,2102,Agustín,I-ORG
14037,5,',O
14038,2092,suyu,I-LOC


# 3 Etiquetado

Nuestro datos anotados cuenta con la siguiente estrucutura (Formato BIO)

*   O, No es una entidad Nombrada.
*   B-PER, Comienzo del nombre de una persona justo después del nombre de otra persona.
*   B-PER, Comienzo del nombre de una persona justo después del nombre de otra persona.
*   I-PER, Nombre de la persona.
*   B-ORG, Comienzo del nombre de una organización justo después del nombre de otra organización.
*   I-ORG, Organización.
*   B-LOC, Comienzo del nombre de una locación justo después del nombre de otra locación.
*   I-LOC, Locación.


In [ ]:
label = test_data["labels"].unique().tolist()
label

['I-LOC', 'I-ORG', 'O', 'B-LOC', 'B-PER', 'I-PER', 'B-ORG']

# 4 Fine-Tuning Model

La libreria SimpleTransformers,nos permitira hacer el fine tuning para el NER ,en donde la documentacion se muestra aqui: [model](https://simpletransformers.ai/docs/ner-model/)

In [ ]:
from simpletransformers.ner import NERModel,NERArgs

Abajo esta nuestro hiper-parametros correspondients


In [ ]:
args = NERArgs()
args.num_train_epochs = 10
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32

Importamos el modelo pre entrenado del quechua, que se encuentra en el siguiente [enlace](https://github.com/ctoshio/bert-quechua).

El modelo pre entrenado debe comprender los siguientes archivos:

*   config.json
*   merges
*   pytorch_model.bin
*   tokenizer_config.json
*   training_args.bin
*   vocab.json

Donde la carpeta QuechuaBPE contiene los archivos mencionado.

In [ ]:
model= NERModel('roberta', "model/")

Comenzamos a entrenar

Evaluamos nuestro modelo Ner para el quechua!

In [ ]:
result, model_outputs, preds_list = model.eval_model(test_data)

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/80 [00:00<?, ?it/s]

Miramos el desempeño del modelo NER

In [ ]:
result

{'eval_loss': 1.371584689244628,
 'f1_score': 0.7360417119270042,
 'precision': 0.7330160103851147,
 'recall': 0.7390924956369983}

Es hora predecir las entidades en quechua!

In [ ]:
prediction, model_output = model.predict(["Titiqaqa qucha, Buliwya"])
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'Titiqaqa': 'B-LOC'}, {'qucha,': 'I-LOC'}, {'Buliwya': 'I-LOC'}]]

In [ ]:
prediction, model_output = model.predict(["Mama Llaqtap San Markus Kuraq Yachay Sunturnin"])
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'Mama': 'B-ORG'},
  {'Llaqtap': 'I-ORG'},
  {'San': 'I-ORG'},
  {'Markus': 'I-ORG'},
  {'Kuraq': 'I-ORG'},
  {'Yachay': 'I-ORG'},
  {'Sunturnin': 'I-ORG'}]]

In [ ]:
prediction, model_output = model.predict(["Lima"])
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'Lima': 'B-LOC'}]]

In [ ]:
prediction, model_output = model.predict(["llaqtaqa Piruw mama llaqtapi Lima llaqta suyupas uma llaqtanmi."])
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'llaqtaqa': 'O'},
  {'Piruw': 'B-ORG'},
  {'mama': 'I-ORG'},
  {'llaqtapi': 'O'},
  {'Lima': 'B-ORG'},
  {'llaqta': 'O'},
  {'suyupas': 'I-ORG'},
  {'uma': 'O'},
  {'llaqtanmi.': 'I-ORG'}]]

In [ ]:
prediction, model_output = model.predict(["Valentinam runasimi rimaq Elvirawan Andawaylaspi riman"]) 
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'Valentinam': 'B-PER'},
  {'runasimi': 'O'},
  {'rimaq': 'O'},
  {'Elvirawan': 'B-PER'},
  {'Andawaylaspi': 'I-ORG'},
  {'riman': 'I-PER'}]]

In [ ]:

prediction, model_output = model.predict(["Manam Limatachu, Ariqipa ripukun."]) 
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'Manam': 'O'},
  {'Limatachu,': 'B-LOC'},
  {'Ariqipa': 'B-LOC'},
  {'ripukun.': 'I-PER'}]]

In [ ]:
#Chakrapi llamkagchu Teofilo Andawaylasta rin?
prediction, model_output = model.predict(["Chakrapi llamkagchu Teofilo Andawaylasta rin?"]) 
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'Chakrapi': 'O'},
  {'llamkagchu': 'O'},
  {'Teofilo': 'O'},
  {'Andawaylasta': 'O'},
  {'rin?': 'O'}]]

In [ ]:
prediction, model_output = model.predict(["Wamangaga Andawaylasmanta ancha sumaqmi"]) 
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'Wamangaga': 'I-LOC'},
  {'Andawaylasmanta': 'B-LOC'},
  {'ancha': 'I-LOC'},
  {'sumaqmi': 'B-LOC'}]]

In [ ]:
# Antes de que termine la cosecha del trigo, llega la gente de Querobamba
prediction, model_output = model.predict(["Manaraq trigu iray tukuchkaptinmi Qirubamba runakuna chayamun"]) 
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'Manaraq': 'I-PER'},
  {'trigu': 'O'},
  {'iray': 'O'},
  {'tukuchkaptinmi': 'O'},
  {'Qirubamba': 'I-PER'},
  {'runakuna': 'O'},
  {'chayamun': 'O'}]]

In [ ]:
#Aquel es el río Rímac.
prediction, model_output = model.predict(["Wakqa Rimaq mayum."]) 
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'Wakqa': 'B-LOC'}, {'Rimaq': 'B-LOC'}, {'mayum.': 'O'}]]

In [ ]:
#Soy de Andahuaylas, por eso hablo quechua
prediction, model_output = model.predict(["Andawaylasmantam kani, chaymi runasimita rimani"]) 
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'Andawaylasmantam': 'B-LOC'},
  {'kani,': 'O'},
  {'chaymi': 'B-LOC'},
  {'runasimita': 'O'},
  {'rimani': 'O'}]]

In [ ]:
#¿La madre de Juan vive en Talavera?
prediction, model_output = model.predict(["Talaverapichu Juanpa maman tiyan?"]) 
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'Talaverapichu': 'I-PER'},
  {'Juanpa': 'B-PER'},
  {'maman': 'I-PER'},
  {'tiyan?': 'O'}]]

In [ ]:
#Silvia está esperando a Pedro en la puerta del cementerio.
prediction, model_output = model.predict(["Silviam Pedrota pantiyun punkupi suyachkan."]) 
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'Silviam': 'I-PER'},
  {'Pedrota': 'B-PER'},
  {'pantiyun': 'O'},
  {'punkupi': 'O'},
  {'suyachkan.': 'I-PER'}]]

In [ ]:
#¡Pero si Mariano tiene una casa en Andahuaylas!
prediction, model_output = model.predict(["Marianoga Andawaylaspi wasiyuqmá!"]) 
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'Marianoga': 'B-PER'}, {'Andawaylaspi': 'I-PER'}, {'wasiyuqmá!': 'I-PER'}]]

In [ ]:
!pip install -q pyngrok
from pyngrok import ngrok

     |████████████████████████████████| 745 kB 5.2 MB/s 


In [26]:
url = ngrok.connect(8501)
url.public_url

'http://d5e7-146-148-49-5.ngrok.io'

In [27]:
!simple-viewer


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://146.148.49.5:8501

2021-10-04 01:33:48.728  Converting to features started.
100% 1/1 [00:00<00:00, 9383.23it/s]
Running Prediction: 100% 1/1 [00:00<00:00, 24.08it/s]
  Stopping...
